In [64]:
# This version assumes the following:
# 1. The target nodes can NOT be interlinked.
# 2. At least arcs between any pair of (true) source-target node.
# 3. The evader cannot see the 2nd-stage interdiction decision
# 4. There is no terminating condition yet

using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs
gurobi_env = Gurobi.Env()
to = TimerOutput()
myFile = "./SmallExample.jl"
include(myFile)
myFile = "./shortestPath.jl"
include(myFile)
b=1
M = 1000#sum(c[t] for t =1:length(T))*10


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


1000

In [65]:
x_now = zeros(Len)
# y, gx, tree, path, y_Set = gx_bound(arcs, Len, c, origin,d, x_now, T, scenNum)
y_Set, g_Set = gx_bound(arcs, Len, c, origin,d, x_now, T, scenNum)

([1 0 … 0 0; 1 0 … 0 0; 0 1 … 1 0], [18.0, 17.0, 14.0])

In [76]:
###Full Formulation:
N2 = [3,4,8]

#Initialize p^n_t to be just replications of p
p = Array{Float64}(undef, 0,length(T))
for n in N2
    p = vcat(p, p_init')
end
# println(p)
# println(p)
# println(Len)
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
FP = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(FP, x1[1:Len], Bin)
@variable(FP, x2[1:length(N2),1:Len], Bin)
@variable(FP, z[1:length(N2), 1:length(T)] >=0)

constr = Array{JuMP.ConstraintRef}(undef, 200)
# println("!")
@objective(FP, Min, sum(p[n,t]*z[n,t] for n = 1:length(N2) for t=1:length(T) ) )

#Budget Constraints
for n = 1:length(N2)
    @constraint(FP, sum(x2[n,i] for i=1:Len) <= b - sum(x1[i] for i =1:Len))
end

#Interdiction Constraints on Super-node Arcs:
for i in A_star
    @constraint(FP, x1[i] == 0)
    for n = 1:length(N2) #scenNum
        @constraint(FP, x2[n,i] == 0)
    end
end
opt = false
iter = 0
con_num = 1
last_Obj = -1
while opt == false
    iter = iter+1
    println("\n\nIteration ", iter)
#     println(FP)
    optimize!(FP)
    current_Obj = JuMP.objective_value.(FP)
    current_x1 = JuMP.value.(x1)
    current_x2 = JuMP.value.(x2)
    current_z = JuMP.value.(z)
    println("Obj Val = ", current_Obj)
    println("x1 = ", findall(current_x1.==1))
    for n = 1: length(N2)#scenNum
        println("n = ", N2[n], ". x2 = ", findall(current_x2[n,:].==1))
        println("n = ", N2[n], ". z = ", current_z[n,:])
    end
    for n = 1:length(N2)
        arc_Set = findall(arcs[:,2].== N2[n])
#         println(arc_Set)
        
        #Solve for shortest path:
        y_All, g_All = gx_bound(arcs, Len, c, N2[n], d, current_x1, T, scenNum)
        println(g_All)
        p_sum = 0
        for t = 1:length(T)
            #Constraint for impact of stage 1's interdiction decision on z^n_t
            constr[con_num] = @constraint(FP, z[n,t] <= M*(sum(x1[i] for i in arc_Set)))
            con_num = con_num +1
            
            if g_All[t] <= Len
#                 y_Set = vcat(y_Set,y_All[n,:]')
                evaderPath = findall(y_All[n,:].==1)
                constr[con_num] = @constraint(FP, z[n,t] >= v[t]*(1 - sum(x2[n,i] for i in evaderPath)))
                con_num = con_num +1
                p_sum = p_sum + p[n,t]
            else 
                p[n,t] = 0
            end
            
        end
        p[n,:] = p[n,:]./p_sum
#         println("p[n,:] = ", p[n,:])
    end
    if last_Obj >= current_Obj
        opt = true
    end
    last_Obj = current_Obj
    @objective(FP, Min, sum(p[n,t]*z[n,t] for n = 1:length(N2) for t=1:length(T) ) )
end





Iteration 1
Obj Val = 0.0
x1 = Int64[]
n = 3. x2 = Int64[]
n = 3. z = [0.0, 0.0, 0.0]
n = 4. x2 = Int64[]
n = 4. z = [0.0, 0.0, 0.0]
n = 8. x2 = Int64[]
n = 8. z = [0.0, 0.0, 0.0]
[1.0, 14.0, 14.0]
[14.0, 10.0, 14.0]
[14.0, 8.0, 2.0]


Iteration 2
Obj Val = 0.0
x1 = Int64[]
n = 3. x2 = [5]
n = 3. z = [0.0, 0.0, 0.0]
n = 4. x2 = [6]
n = 4. z = [0.0, 0.0, 0.0]
n = 8. x2 = [12]
n = 8. z = [0.0, 0.0, 0.0]
[1.0, 14.0, 14.0]
[14.0, 10.0, 14.0]
[14.0, 8.0, 2.0]
